# Parsing XML with Python

Python provides many libraries for working with xml files such as 

- **`lxml`**: It is a clean, fast and strict library for dealing with xml fils. It's also the most accepted library. It also supports `xpath` and `xslt`.

- **`BeautifulSoup`**: It is flexible but a bit slower than `lxml`. The good thing is if your xml markup is messed up, it will try to correct it. It's perfect for dealing with web scrapped data in HTML formats. For clean xml, it might be too slow. It has been discussed in details in chapter `Chapter S2.05 - REST API - Server & Clients`

- **`xml`** : It has native integration in Python and is fast & clean but do not support xpath and xslt. We will discuss about it more in details in this chapter

Read about others on the Python [official wiki](https://wiki.python.org/moin/PythonXml)

## `lxml`

Based on my experience, `lxml` will meet most of our needs in handling clean data. Here **"`Clean`"** is the keyword. It will not be able to properly handle invalid `html` or `xml`. It will just throw error message.

> This is a third party library and thus needs to be installed using `pip`

### From file to XML object

Opening an xml file is actually quite simple : you open it and you parse it. Who would have guessed ?

In [2]:
from lxml import etree

with open("code/data/books.xml") as fxml:    
    parsed = etree.parse(fxml)

print(parsed, dir(parsed))

<lxml.etree._ElementTree object at 0x7fcbd81ad4c0> ['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_setroot', 'docinfo', 'find', 'findall', 'findtext', 'getelementpath', 'getiterator', 'getpath', 'getroot', 'iter', 'iterfind', 'parse', 'parser', 'relaxng', 'write', 'write_c14n', 'xinclude', 'xmlschema', 'xpath', 'xslt']


As you can see, we obtained an instance of type `lxml.etree._ElementTree`

In [3]:
# New etree parser, with empty text nodes removed

parser = etree.XMLParser(remove_blank_text=True)

with open("code/data/books.xml") as file:
    parsed = etree.parse(file, parser)

print(parsed)

Few useful arguments of `XMLParser` are as follows:

- *attribute_defaults* : Use DTD (if available) to add the default attributes
- *dtd_validation* : Validate against DTD while parsing
- *load_dtd* : Load and parse the DTD while parsing
- *ns_clean* : Clean up redundant namespace declarations
- *recover* : Try to fix ill-formed xml
- *remove_blank_text* : Removes blank text nodes
- *resolve_entities* : Replace entities by their value (Default : on)

You can then create a new parser according to its standards or clean namespace attribute. In this context, *ns_clean* would transform

`<root xmlns:a="xmlns1" xmlns:b="xmlns2">
    <tag xmlns:c="xmlns3" />
    <tag xmlns:a="xmlns1" />
    <tag />
</root>`

into

`<root xmlns:a="xmlns1" xmlns:b="xmlns2">
    <tag xmlns:c="xmlns3" />
    <tag />
    <tag />
</root>`

### Validating Against a DTD

In [4]:
from lxml import etree
parser = etree.XMLParser(dtd_validation=True)
tree = etree.parse("code/data/addresses_with_dtd.xml", parser)

In [9]:
from lxml import etree
parser = etree.XMLParser(dtd_validation=True)
try:
    tree = etree.parse("code/data/addresses_with_dtd_e1.xml", parser)
except Exception as e:
    print(e)

No declaration for element ph_no, line 15, column 31 (addresses_with_dtd_e1.xml, line 15)


In [14]:
# Error suppressed by `dtd_validate=False` which is default behavior.

from lxml import etree
parser = etree.XMLParser(dtd_validation=False)
try:
    tree = etree.parse("code/data/addresses_with_dtd_e1.xml", parser)
    print(tree)
except Exception as e:
    print(e)

In [8]:
from lxml import etree
parser = etree.XMLParser(dtd_validation=True)
try:
    tree = etree.parse("code/data/addresses_with_dtd_e2.xml", parser)
    print(tree)
except Exception as e:
    print(e)

### String to XML object

`lxml` parses `strings` using `fromstring` function which is similar to `parse` which is used to parses files as shown in the below example. 

In [3]:
xml = '<root xmlns:a="xmlns1" xmlns:b="xmlns2"><tag xmlns:c="xmlns3" /><tag xmlns:a="xmlns1" /><tag /></root>'
parsed = etree.fromstring(xml)
print(parsed)

<Element root at 0x7f46c8141108>


### Errors handling

As previously stated `lxml` is quite strict about `xml` validity. Let's try following example to understand it in details.
Below we have three invalid xml strings, lets see how they are handled by `lxml`

In [3]:
from lxml.etree import XMLSyntaxError
xmls = ["""
""",
        
"""
<start>Lets have errors</start>
<start>this is a text</start1>
""",
        """
<start attr="test">
""",
        """
        <root>
<start attr="test"/>
</root>
"""
       ]
for x in xmls:
    try:
        print("*** XML:", x)
        etree.fromstring(x)
    except XMLSyntaxError as e:
        print(e)

*** XML: 

Start tag expected, '<' not found, line 2, column 1 (<string>, line 2)
*** XML: 
<start>Lets have errors</start>
<start>this is a text</start1>

Extra content at the end of the document, line 3, column 1 (<string>, line 3)
*** XML: 
<start attr="test">

EndTag: '</' not found, line 3, column 1 (<string>, line 3)
*** XML: 
        <root>
<start attr="test"/>
</root>



As we can see, most of the errors are detailed enough so we can correct the XML file manually.

### Node properties and methods

> **!! Warning : namespaces !!** : 
> ----
> In lxml, namespaces are expressed using the Clark notation. This mean that, if a namespace defines a node, this node will be named using the following syntax "`{namespace}tagname`. Here is an example :

In [5]:
# With no namespace
print(etree.fromstring("<root />"))
# With namespace
print(etree.fromstring("<root xmlns='http://localhost' />"))

<Element root at 0x7f46c80cf0c8>
<Element {http://localhost}root at 0x7f46c80a1948>


Below is the chart detailing the functionalities offered by `lxml`

![Cheatsheet](images/CheatsheetElement.svg)

Let's see what that means in real life :

In [17]:
# First, we will need some xml
xml = """
<div type="Book" n="1">
    <l n="1">The Rigveda is undoubtedly the oldest literary monument of the Indo-European languages.</l>
    <mj:l n="2" xmlns:mj="http://www.mayankjohri.org/ns/1.0">But the exact period when the hymns were composed is a matter of conjecture.</mj:l>
    <l n="3">All that we can say with any approach to certainty is that the oldest of them cannot </l>
    <l n="4">date from later than the thirteenth century bc.</l>
    <l n="5">This assertion is based on the following grounds. B</l>
    <l n="6">uddhism, which began to spread in India about 500 bc, </l>
    <l n="7">presupposes the existence not only of the Vedas, </l>
</div>
"""
# Reading the xml string into etree.
book = etree.fromstring(xml)
print(book)

<Element div at 0x10b7916c0>


If we want to retrieve the attributes of our div, we can do as follow :

In [24]:
print(book.get("type"))
# or
print(book.get("type", None))
# or
print(book.attrib["type"])

Book
Book
Book


In [25]:
# Just like dictionary
print(book.get("type1", None))

None


In [27]:
print(book.attrib)
print(book.items())

{'type': 'Book', 'n': '1'}
[('type', 'Book'), ('n', '1')]


In [30]:
for attr in book.attrib:
    print(attr, "=>", book.get(attr))

type => Book
n => 1


Lets get more details from the xml file, we can use the following to get them :

- `getchildren()` will returns a list of children tags, such as div.
- list(`book`) will transform `book` in a list of sentences.

Both syntaxes return the same results, so it's up to you to decide which one you prefer. 

In [36]:
from pprint import pprint

children = book.getchildren()
pprint(children)

[<Element l at 0x10b787680>,
 <Element {http://www.mayankjohri.org/ns/1.0}l at 0x10b787840>,
 <Element l at 0x10b787480>,
 <Element l at 0x10b787880>,
 <Element l at 0x10b787c40>,
 <Element l at 0x10b787b40>,
 <Element l at 0x10b787bc0>]


In [35]:
# or process them one at a time
for ele in book.getchildren():
    print(ele)

<Element l at 0x10b787680>
<Element {http://www.mayankjohri.org/ns/1.0}l at 0x10b787840>
<Element l at 0x10b787480>
<Element l at 0x10b787880>
<Element l at 0x10b787c40>
<Element l at 0x10b787b40>
<Element l at 0x10b787bc0>


To get one element from the list of elements,

In [33]:
line_1 = children[0] 
pprint(line_1)

<Element l at 0x10b787680>


Now that we have access to our children, we can have access to their text :

In [38]:
print(line_1.text)

The Rigveda is undoubtedly the oldest literary monument of the Indo-European languages.


In [40]:
for ele in book.getchildren():
    print(ele.text, ele.items())

The Rigveda is undoubtedly the oldest literary monument of the Indo-European languages. [('n', '1')]
But the exact period when the hymns were composed is a matter of conjecture. [('n', '2')]
All that we can say with any approach to certainty is that the oldest of them cannot  [('n', '3')]
date from later than the thirteenth century bc. [('n', '4')]
This assertion is based on the following grounds. B [('n', '5')]
uddhism, which began to spread in India about 500 bc,  [('n', '6')]
presupposes the existence not only of the Vedas,  [('n', '7')]


Ok, we are now able to get some stuff done. Remember the namespace naming ? Sometimes it's useful to retrieve namespaces and their prefix :

In [41]:
line_2 = children[1]
print(line_2.nsmap)
print(line_2.prefix)
print(line_2.tag)

{'mj': 'http://www.mayankjohri.org/ns/1.0'}
mj
{http://www.mayankjohri.org/ns/1.0}l


In [48]:
for ele in book.getchildren():
    print(ele.text)

The Rigveda is undoubtedly the oldest literary monument of the Indo-European languages.
But the exact period when the hymns were composed is a matter of conjecture.
All that we can say with any approach to certainty is that the oldest of them cannot 
date from later than the thirteenth century bc.
This assertion is based on the following grounds. B
uddhism, which began to spread in India about 500 bc, 
presupposes the existence not only of the Vedas, 


In [42]:
for ele in book.getchildren():
    print(ele.prefix, ele.tag.upper(), ele.attrib)

None L {'n': '1'}
mj {HTTP://WWW.MAYANKJOHRI.ORG/NS/1.0}L {'n': '2'}
None L {'n': '3'}
None L {'n': '4'}
None L {'n': '5'}
None L {'n': '6'}
None L {'n': '7'}


**What you've learned** :

- How to parse a xml file or a string representing xml through `etree.parse()` and `etree.fromstring()`
- How to configure the way xml is parsed with `etree.XMLParser()`
- What is an attribute and a method
- Properties and methods of a node
- XMLParseError handling
- Clark's notation for namespaces and tags.

### XPath and XSLT with lxml

#### XPath

XPath is a powerful tool for traversing an xml tree. XML is made of nodes such as tags, comments, texts. These nodes have attributes that can be used to identify them. For example, with the following xml :

> `<div><l n="1"><p>Text</p> followed</l><l n="2">by line two</div>`

the node p will be accessible by `/div/l[@n="1"]/p`. LXML has great support for complex XPath, which makes it the best friend of Humanists dealing with xml :

In [4]:
# We generate some xml and parse it

## TODO 
xml = """<div>
            <cities n="1">
                <p>Chennai</p> 
                <p>Agra</p>
                followed
                <test>
                    <p>p3</p>
                </test>
            </cities>
            <cities n="2">
                <p>Vish</p>
                <subcity>
                    <p>Navel Base</p>
                </subcity>
            </cities>
            <p>test</p>
            <p><cities n="3"><p>Bhopal</p></cities></p>
        </div>"""
div = etree.fromstring(xml)
print(div)
# When doing an xpath, the results will be a list
print("-"*20)
ps = div.xpath("/div/cities[@n='1']/p")

<Element div at 0x7fcbc8e28c80>
--------------------


In [5]:
for p in ps:
    print(p, p.text)

<Element p at 0x7fcbc8e32e80> Chennai
<Element p at 0x7fcbc8e28780> Agra


As you can see, the xpath returns a list. This behaviour is intended, since an xpath can retrieve more than one item :

In [56]:
print(div.xpath("//cities"))

[<Element cities at 0x10b7b8c00>, <Element cities at 0x10b7b8640>, <Element cities at 0x10b7b8600>]


In [86]:
for pt in div.xpath("//cities[@n=2]/p"):
    print(pt.text)

Vish


In [ ]:
for pt in div.xpath("//cities/p"):
    print(pt.text)

You see ? The xpath `//l` returns two elements, just like python does in a list. Now, let's apply some xpath to the children and see what happens :

In [97]:
for pt in div.xpath("p"):
    print(pt.text)

test
None


In [60]:
# We look for p

line_1 = div.xpath("//cities")  # It will return a list
for a in line_1[1]:
    print(a.text)


Vish

                    


In [61]:
# child Element Only
print(line_1[1].xpath("p"))
print(line_1[1].xpath("./p"))

# All childern nodes & subnodes with XPath specified.
print(line_1[1].xpath(".//p"))

[<Element p at 0x10b782780>]
[<Element p at 0x10b782a00>]
[<Element p at 0x10b782180>, <Element p at 0x10b782a00>]


In [62]:

for pt in line_1[1].xpath(".//p"):
    print(pt.text)

Vish
Navel Base


As you can see, you can do xpath from any node in lxml. One important thing though : xpath `//tagname` *will return to the root* if you do not add a dot in front of it such as **`.`**`//tagname`. This is really important to remember, because most xpath resolvers do not behave this way.

Another point to kepe in mind : if you write your xpath incorrectly, Python will raise an *XPathEvalError * error

In [63]:
from lxml.etree import XPathEvalError
try:
    line_1[1].xpath("wrong:xpath:never:works")
except XPathEvalError as e:
    print(e.__str__())

Invalid expression


### Xpath with namespaces and prefix

As you've seen, lxml use Clark's naming convention for expressing namespaces. This is extremely important regarding xpath, because you will be able to retrieve a node using it under certain conditions :

In [16]:
# We create a valid xml object
xml = """<root>
<tag xmlns="http://localhost">Text</tag>
<tei:tag xmlns:tei="http://www.tei-c.org/ns/1.0">Other text</tei:tag>
<teiTwo:tag xmlns:teiTwo="http://www.tei-c.org/ns/2.0">Other text</teiTwo:tag>
</root>"""
root = etree.fromstring(xml)
# We register every namespaces in a dictionary using prefix as keys :
ns = {
    "local" : "http://localhost", # Even if this namespace had no prefix, we can register one for it
    "tei" : "http://www.tei-c.org/ns/1.0",
    "two": "http://www.tei-c.org/ns/2.0"
}

print([d.text for namespace in ns 
       for d in root.xpath("//{namespace}:tag".format(namespace=namespace), 
                           namespaces=ns) ])

['Text', 'Other text', 'Other text']


**What you have learned** :

- Each node and xml document has an `.xpath()` method which takes as its first parameter xpath
- Method `xpath()` always returns a list, even for a single result
- Method `xpath()` will return to the root when you don't prefix your `//` with a dot.
- An incorrect XPath will issue a `XPathEvalError`
- Method `xpath()` accepts a `namespaces` argument : you should enter a dictionary where keys are prefixes and values namespaces
- Unlike `findall()`, `xpath()` does not accept Clark's notation

### XSLT

XSLT stands for *Extensible Stylesheet Language Transformations*. It's an xml-based language made for transforming xml documents to xml or other formats such as LaTeX and HTML. XSLT is really powerful when dealing with similarly formated data. It's far easier to transform 100 documents with the exact same structure via XSLT than in Python or any other language.

While Python is great at dealing with weird transformations of xml, the presence of XSLT in Python allows you to create production chains without leaving your favorite IDE.

To do some XSL, lxml needs two things : first, an xml document representing the xsl that will be parsed and entered into the function `etree.XSLT()`, and second, a document to transform.

In [64]:
# Here is an xml containing an xsl: for each text node of an xml file in the xpath /humanities/field,
#     this will return a node <name> with the text inside
xslt_root = etree.fromstring("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:template match="/">
        <fields><xsl:apply-templates /></fields>
    </xsl:template>
    <xsl:template match="/humanities/field">
        <name><xsl:value-of select="./text()" /></name>
    </xsl:template>
</xsl:stylesheet>""")
# We transform our document to an xsl 
xslt = etree.XSLT(xslt_root)

# We create some xml we need to change 
xml = """<humanities>
    <field>History</field>
    <field>Classics</field>
    <field>French</field>
    <field>German</field>
</humanities>"""
parsed_xml = etree.fromstring(xml)
# And now we process our xml :
transformed = xslt(parsed_xml)
print(transformed)

<?xml version="1.0"?>
<fields>
    <name>History</name>
    <name>Classics</name>
    <name>French</name>
    <name>German</name>
</fields>



Did you see what happened ? We used `xslt(xml)`. `etree.XSLT()` transforms a xsl document into a function, which then takes one parameter (in this case an xml document). But can you figure out what this returns ? Let's ask Python :

In [18]:
print(type(transformed))
print(type(parsed_xml))

<class 'lxml.etree._XSLTResultTree'>
<class 'lxml.etree._Element'>


The result is not of the same type of element we usually have, even though it does share most of its methods and attributes :

In [19]:
print(transformed.xpath("//name"))

[<Element name at 0x7f46c8063488>, <Element name at 0x7f46c8063648>, <Element name at 0x7f46c8063208>, <Element name at 0x7f46c80635c8>]


And has something more : you can change its type to string !

In [20]:
string_result = str(transformed)
print(string_result)

<?xml version="1.0"?>
<fields>
    <name>History</name>
    <name>Classics</name>
    <name>French</name>
    <name>German</name>
</fields>



XSLT is more complex than just inputing xml. You can do XSLT using parameters as well. In this case, your parameters will be accessibles as a named argument to the generated function. If your XSL has a `name` xsl-param, the function given back by `etree.XSLT` will have a `name` argument :

In [21]:
# Here is an xml containing an xsl: for each text node of an xml file in the xpath /humanities/field,
#     this will return a node <name> with the text inside
xslt_root = etree.fromstring("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:param name="n" />
    <xsl:template match="/humanities">
        <fields>
            <xsl:attribute name="n">
                <xsl:value-of select="$n"/>
            </xsl:attribute>
            <xsl:apply-templates select="field"/>
        </fields>
    </xsl:template>
    <xsl:template match="/humanities/field">
        <name><xsl:value-of select="./text()" /></name>
    </xsl:template>
</xsl:stylesheet>""")
# We transform our document to an xsl 
xslt = etree.XSLT(xslt_root)

# We create some xml we need to change 
xml = """<humanities>
    <category>Humanities</category>
    <field>History</field>
    <field>Classics</field>
    <field>French</field>
    <field>German</field>
</humanities>"""
parsed_xml = etree.fromstring(xml)
# And now we process our xml :
transformed = xslt(parsed_xml, n="'Humanities'") # Note that for a string, we encapsulate it within single quotes
print(transformed)

# Be aware that you can use xpath as a value for the argument, though it can be rather complex sometimes
transformed = xslt(parsed_xml, n=etree.XPath("//category/text()"))
print(transformed)

<?xml version="1.0"?>
<fields n="Humanities"><name>History</name><name>Classics</name><name>French</name><name>German</name></fields>

<?xml version="1.0"?>
<fields n="Humanities"><name>History</name><name>Classics</name><name>French</name><name>German</name></fields>



### Using ElementTree

In [6]:
from xml.etree import ElementTree

with open('code/data/books.xml', 'rt') as f:
    tree = ElementTree.parse(f)

print(tree)

### Traversing the Parsed Tree

To visit all of the children in order, use iter() to create a generator that iterates over the ElementTree instance.

In [7]:
from xml.etree import ElementTree

    
with open('code/data/books.xml', 'r') as f:
    tree = ElementTree.parse(f)

In [8]:
for node in tree.iter():
    print (node.tag, node.attrib)
    print("-----")

catalog {}
-----
book {'id': 'bk101'}
-----
author {}
-----
title {}
-----
genre {}
-----
book {'part': '2'}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk102'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk103'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk104'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk105'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk106'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk107'}
-----
author {}
-----
title {}
-----
genre {}
-----
price {}
-----
publish_date {}
-----
description {}
-----
book {'id': 'bk108'}
-----
author {}
----

In [9]:
# only getting 5 elements from the generator
from itertools import islice

for node in islice(tree.iter(), 5):
    print (node.tag, node.attrib)
    print("-----")

catalog {}
-----
book {'id': 'bk101'}
-----
author {}
-----
title {}
-----
genre {}
-----


In [72]:
### To print only the groups of names and feed URLs for the podcasts, 
# leaving out of all of the data in the header section by iterating 
# over only the outline nodes and print the text and xmlUrl attributes.

from xml.etree import ElementTree

with open('code/data/podcasts.opml', 'rt') as f:
    tree = ElementTree.parse(f)

print(len( list(tree.iter('outline'))))
for node in tree.iter('outline'):
    name = node.attrib.get('text')
    url = node.attrib.get('xmlUrl')
    if name and url:
        print ('\t%s :: %s' % (name, url))
    else:
        print (name)

18
Science and Tech
	APM: Future Tense :: http://www.publicradio.org/columns/futuretense/podcast.xml
	Engines Of Our Ingenuity Podcast :: http://www.npr.org/rss/podcast.php?id=510030
	Science & the City :: http://www.nyas.org/Podcasts/Atom.axd
Books and Fiction
	Podiobooker :: http://feeds.feedburner.com/podiobooks
	The Drabblecast :: http://web.me.com/normsherman/Site/Podcast/rss.xml
	tor.com / category / tordotstories :: http://www.tor.com/rss/category/TorDotStories
Computers and Programming
	MacBreak Weekly :: http://leo.am/podcasts/mbw
	FLOSS Weekly :: http://leo.am/podcasts/floss
	Core Intuition :: http://www.coreint.org/podcast.xml
Python
	PyCon Podcast :: http://advocacy.python.org/podcasts/pycon.rss
	A Little Bit of Python :: http://advocacy.python.org/podcasts/littlebit.rss
	Django Dose Everything Feed :: http://djangodose.com/everything/feed/
Miscelaneous
	dhellmann's CastSampler Feed :: http://www.castsampler.com/cast/feed/rss/dhellmann/


### Finding Nodes in a Document

Walking the entire tree like this searching for relevant nodes can be error prone. The example above had to look at each outline node to determine if it was a group (nodes with only a text attribute) or podcast (with both text and xmlUrl). To produce a simple list of the podcast feed URLs, without names or groups, for a podcast downloader application, the logic could be simplified using findall() to look for nodes with more descriptive search characteristics.

As a first pass at converting the above example, we can construct an XPath argument to look for all outline nodes.

In [73]:
for node in tree.findall('.//outline'):
    url = node.attrib.get('xmlUrl')
    if url:
        print( url)
    else:
        print(node.attrib.get("text"))

Science and Tech
http://www.publicradio.org/columns/futuretense/podcast.xml
http://www.npr.org/rss/podcast.php?id=510030
http://www.nyas.org/Podcasts/Atom.axd
Books and Fiction
http://feeds.feedburner.com/podiobooks
http://web.me.com/normsherman/Site/Podcast/rss.xml
http://www.tor.com/rss/category/TorDotStories
Computers and Programming
http://leo.am/podcasts/mbw
http://leo.am/podcasts/floss
http://www.coreint.org/podcast.xml
Python
http://advocacy.python.org/podcasts/pycon.rss
http://advocacy.python.org/podcasts/littlebit.rss
http://djangodose.com/everything/feed/
Miscelaneous
http://www.castsampler.com/cast/feed/rss/dhellmann/


In [26]:
print(dir(tree))
print(tree.getroot)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_root', '_setroot', 'find', 'findall', 'findtext', 'getiterator', 'getroot', 'iter', 'iterfind', 'parse', 'write', 'write_c14n']
<bound method ElementTree.getroot of <xml.etree.ElementTree.ElementTree object at 0x7f46c8013be0>>


Another version can take advantage of the fact that the outline nodes are only nested two levels deep. Changing the search path to .//outline/outline mean the loop will process only the second level of outline nodes.

In [74]:
for node in tree.findall('.//outline/outline'):
    url = node.attrib.get('xmlUrl')
    print (url)

http://www.publicradio.org/columns/futuretense/podcast.xml
http://www.npr.org/rss/podcast.php?id=510030
http://www.nyas.org/Podcasts/Atom.axd
http://feeds.feedburner.com/podiobooks
http://web.me.com/normsherman/Site/Podcast/rss.xml
http://www.tor.com/rss/category/TorDotStories
http://leo.am/podcasts/mbw
http://leo.am/podcasts/floss
http://www.coreint.org/podcast.xml
http://advocacy.python.org/podcasts/pycon.rss
http://advocacy.python.org/podcasts/littlebit.rss
http://djangodose.com/everything/feed/
http://www.castsampler.com/cast/feed/rss/dhellmann/


### Parsed Node Attributes

The items returned by `findall()` and `iter()` are Element objects, each representing a node in the XML parse tree. Each Element has attributes for accessing data pulled out of the XML. This can be illustrated with a somewhat more contrived example input file, data.xml:

In [75]:
from xml.etree import ElementTree

with open('code/data/data.xml', 'rt') as f:
    tree = ElementTree.parse(f)

node = tree.find('./with_attributes')
print (node.tag)

for name, value in sorted(node.attrib.items()):
    print ('  %-4s = "%s"' % (name, value))

with_attributes
  foo  = "bar"
  name = "value"
  testtest = "thisis atest"


In [76]:
for path in [ './child', './child_with_tail' ]:
    node = tree.find(path)
    print(node.tag)
    print ('  child node text:', node.text)
    print ('  and tail text  :', node.tail)

child
  child node text: This child contains text.
  and tail text  : 
  
child_with_tail
  child node text: This child has regular text.
  and tail text  : And "tail" text.
  


### Parsing Strings

To work with smaller bits of XML text, especially string literals as might be embedded in the source of a program, use XML() and the string containing the XML to be parsed as the only argument.

In [30]:
from xml.etree.ElementTree import XML

parsed = XML('''
<root>
  <group>
    <child id="a">This is child "a".</child>
    <child id="b">This is child "b".</child>
  </group>
  <group>
    <child id="c">This is child "c".</child>
  </group>
</root>
''')

print ('parsed =', parsed)

for elem in parsed:
    print (elem.tag)
    if elem.text is not None and elem.text.strip():
        print ('  text: "%s"' % elem.text)
    if elem.tail is not None and elem.tail.strip():
        print ('  tail: "%s"' % elem.tail)
    for name, value in sorted(elem.attrib.items()):
        print('  %-4s = "%s"' % (name, value))
    print

parsed = <Element 'root' at 0x7f46c8088ef8>
group
group


In [31]:
from xml.etree.ElementTree import Element, tostring

top = Element('top')

children = [
    Element('child', num=str(i))
    for i in range(3)
]

top.extend(children)

print(top)

<Element 'top' at 0x7f46c807cea8>


## XML support in Python

Python has rich support for XML by having multiple libs to parse XML documents. Lets dicuss them in details. Following are the sub-modules supported nativly by Python

- xml.etree.ElementTree: the ElementTree API, a simple and lightweight XML processor
- xml.dom: the DOM API definition
- xml.dom.minidom: a minimal DOM implementation
- xml.dom.pulldom: support for building partial DOM trees
- xml.sax: SAX2 base classes and convenience functions
- xml.parsers.expat: the Expat parser binding

### xml.etree.ElementTree

we can import `ET` using the following command

In [11]:
import xml.etree.ElementTree as ET

XML can parse either the xml file using the following code, 

In [12]:
old_books = 'code/data/old_books.xml'
nasa_data = 'code/data/nasa.xml'

#### Opening xml file

Opening an xml file is actually quite simple : you open it and you parse it. Who would have guessed ?

In [89]:
tree = ET.parse(old_books)
root = tree.getroot()
print(tree)

or read string using the following code

In [14]:
xml_book = """<?xml version="1.0"?>
<books>
    <book title="Ṛg-Veda Khilāni">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2008</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/rvkh/rvkh.htm</web_page>
    </book>    
    <book title="Ṛgveda-Saṃhitā">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2000</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/mt/rv.htm</web_page>
    </book>
</books>
"""
root = ET.fromstring(xml_book)

As an Element, `root` also has tag and to following code can be used to find the tag

In [36]:
print(root.tag)

books


We can use `len` to find the number of direct child nodes. As in our example we have two `book` nodes,  

In [37]:
print(len(root))

2


#### Reading root as binary text

In [38]:
from pprint import pprint
pprint(ET.tostring(root))

(b'<books>\n    <book title="&#7770;g-Veda Khil&#257;ni">\n        <editor>Jo'
 b'st Gippert</editor>\n        <publication>Frankfurt: TITUS</publication>\n'
 b'        <year>2008</year>\n        <web_page>http://titus.uni-frankfurt.d'
 b'e/texte/etcs/ind/aind/ved/rv/rvkh/rvkh.htm</web_page>\n    </book>    \n  '
 b'  <book title="&#7770;gveda-Sa&#7747;hit&#257;">\n        <editor>Jost Gi'
 b'ppert</editor>\n        <publication>Frankfurt: TITUS</publication>\n     '
 b'   <year>2000</year>\n        <web_page>http://titus.uni-frankfurt.de/tex'
 b'te/etcs/ind/aind/ved/rv/mt/rv.htm</web_page>\n    </book>\n</books>')


#### Reading element as formatted text

In [39]:
dec_root = ET.tostring(root).decode()
print(dec_root)
print(type(dec_root))

<books>
    <book title="&#7770;g-Veda Khil&#257;ni">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2008</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/rvkh/rvkh.htm</web_page>
    </book>    
    <book title="&#7770;gveda-Sa&#7747;hit&#257;">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2000</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/mt/rv.htm</web_page>
    </book>
</books>
<class 'str'>


#### All attributes available to an element

we can use our time tested and trusted  `dir` to get the list of all the attributes of an `element`

In [40]:
print(dir(root))

['__class__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'attrib', 'clear', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'insert', 'items', 'iter', 'iterfind', 'itertext', 'keys', 'makeelement', 'remove', 'set', 'tag', 'tail', 'text']


we can use `for` loop to traverse the direct descendents nodes.

In [41]:
for ele in root:
    print(ele)

<Element 'book' at 0x7f46c8012778>
<Element 'book' at 0x7f46c8086138>


as shown above we get element nodes using for loop, lets get more information from them by enhancing the existing code

In [42]:
for ele in root:
    print(ele.tag, ele.attrib)

book {'title': 'Ṛg-Veda Khilāni'}
book {'title': 'Ṛgveda-Saṃhitā'}


we can also find the nodes using indexes. 

In [43]:
print(root[1])

<Element 'book' at 0x7f46c8086138>


If more than one attibutes are present then individual attributes can be accessas similar to dictionary

In [44]:
print(root[1].attrib['title'])

Ṛgveda-Saṃhitā


In [45]:
print(root[0][1].text)

Frankfurt: TITUS


#### Reading Large XML file using `iterparse`

In [46]:
for event, elem in ET.iterparse(old_books):
    print(event, elem)

end <Element 'editor' at 0x7f46c80169a8>
end <Element 'publication' at 0x7f46c80169f8>
end <Element 'year' at 0x7f46c8016a48>
end <Element 'web_page' at 0x7f46c8016a98>
end <Element 'book' at 0x7f46c8016958>
end <Element 'editor' at 0x7f46c8016b38>
end <Element 'publication' at 0x7f46c8016b88>
end <Element 'year' at 0x7f46c8016bd8>
end <Element 'web_page' at 0x7f46c8016c28>
end <Element 'book' at 0x7f46c8016ae8>
end <Element 'books' at 0x7f46c8016908>


In [137]:
file_name = 'code/data/nasa.xml'
x = 0
for event, elem in ET.iterparse(file_name):
    print("Event:", event, "Elem:", elem)
    if x > 10:
        pass
        break
    else:
        x += 1

Event: end Elem: <Element 'title' at 0x10dece7c8>
Event: end Elem: <Element 'altname' at 0x10dece138>
Event: end Elem: <Element 'altname' at 0x10dece098>
Event: end Elem: <Element 'altname' at 0x10dece188>
Event: end Elem: <Element 'title' at 0x10dec0598>
Event: end Elem: <Element 'initial' at 0x10deeb2c8>
Event: end Elem: <Element 'initial' at 0x10deeb958>
Event: end Elem: <Element 'lastName' at 0x10deeb408>
Event: end Elem: <Element 'author' at 0x10deeb1d8>
Event: end Elem: <Element 'initial' at 0x10deebc78>
Event: end Elem: <Element 'lastName' at 0x10deebea8>
Event: end Elem: <Element 'author' at 0x10deeb0e8>


### Finding interesting elements

Most of the time, we are only interested in part of the whole xml document or only one attribues of all elements.
In this section, we will discuss technologies which will help us in solving the above situations.

Lets assume, our manager (say **Mr. Pauly**) has asked us to read `old_book.xml` file and find the name of editors of all the books. We can solve the request using any of the below methods. 

#### Using iter

This is the **recommemded method** as its an iterator, thus will consume less memory

In [48]:
for editor in root.iter('editor'):
    print(editor)
    print(editor.text)

<Element 'editor' at 0x7f46c8012ea8>
Jost Gippert
<Element 'editor' at 0x7f46c8086188>
Jost Gippert


as, you can see we were able to directly select editor tags

#### using findall

It finds only elements with a tag which are direct children of the current element.

In [49]:
for editor in root.findall('book'):
    print(editor)
    print(editor.tag)

<Element 'book' at 0x7f46c8012778>
book
<Element 'book' at 0x7f46c8086138>
book


In [50]:
print(root.findall('editor'))

[]


As you can see that `editor` is not direct children for the current element `root`, thus we got empty value. But we can use relative xpath to get the editors

In [51]:
print(root.findall('.//editor'))
for editor in root.findall('.//editor'):
    print(editor.tag, " : ", editor.text)

[<Element 'editor' at 0x7f46c8012ea8>, <Element 'editor' at 0x7f46c8086188>]
editor  :  Jost Gippert
editor  :  Jost Gippert


#### Using find

It find the first child with a particular tag

In [52]:
print(root.find('book'))

<Element 'book' at 0x7f46c8012778>


In [53]:
print(root.find('editor'))

None


As you can see that `editor` is not direct children for the current element `root`, thus we got empty value. But we can use relative xpath to get the editors as shown in the previous example.

> <center>! ! **NOTE** ! !</center>
> ---
> Do try to avoid `find` and `findall` as they are not iterators and can & will consume more memory :)

### Accessing Element Attributes

In [54]:
ele = root.find('book')
ele.get('title')

'Ṛg-Veda Khilāni'

### Building XML documents

We can build a XML document using `Element` & `SubElement` functions of `ElementTree`

In [55]:
a = ET.Element('a')
b = ET.SubElement(a, 'b')
b.attrib["B"] = "TEST"
c = ET.SubElement(a, 'c')
d = ET.SubElement(a, 'd')
e = ET.SubElement(d, 'e')
f = ET.SubElement(e, 'f')

ET.dump(a)
print(ET.tostring(a).decode())

<a><b B="TEST" /><c /><d><e><f /></e></d></a>
<a><b B="TEST" /><c /><d><e><f /></e></d></a>


### Parsing XML with Namespaces

```xml
<?xml version="1.0"?>
<actors xmlns:fictional="http://characters.example.com"
        xmlns="http://people.example.com">
    <actor>
        <name>John Cleese</name>
        <fictional:character>Lancelot</fictional:character>
        <fictional:character>Archie Leach</fictional:character>
    </actor>
    <actor>
        <name>Eric Idle</name>
        <fictional:character>Sir Robin</fictional:character>
        <fictional:character>Gunther</fictional:character>
        <fictional:character>Commander Clement</fictional:character>
    </actor>
</actors>
```

In [83]:
xml_text = """<?xml version="1.0"?>
<actors xmlns:fictional="http://characters.example.com"
        xmlns="http://people.example.com">
    <actor>
        <name>John Cleese</name>
        <fictional:character>Lancelot</fictional:character>
        <fictional:character>Archie Leach</fictional:character>
    </actor>
    <actor>
        <name>Eric Idle</name>
        <fictional:character>Sir Robin</fictional:character>
        <fictional:character>Gunther</fictional:character>
        <fictional:character>Commander Clement</fictional:character>
    </actor>
</actors>"""

In [57]:
root = ET.fromstring(xml_text)
for actor in root.findall('{http://people.example.com}actor'):
    name = actor.find('{http://people.example.com}name')
    print(name.text)
    for char in actor.findall('{http://characters.example.com}character'):
        print('   |->', char.text)

John Cleese
   |-> Lancelot
   |-> Archie Leach
Eric Idle
   |-> Sir Robin
   |-> Gunther
   |-> Commander Clement


### XPath support

| Syntax            | Meaning                                                                                                                                                                                                                                   |
|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| tag               | Selects all child elements with the given tag. For example, spam selects all child elements named spam, and spam/egg selects all grandchildren named egg in all children named spam.                                                      |
| *                 | Selects all child elements. For example, */egg selects all grandchildren named egg.                                                                                                                                                       |
| .                 | Selects the current node. This is mostly useful at the beginning of the path, to indicate that it’s a relative path.                                                                                                                      |
| //                | Selects all subelements, on all levels beneath the current element. For example, .//egg selects all eggelements in the entire tree.                                                                                                       |
| ..                | Selects the parent element. Returns None if the path attempts to reach the ancestors of the start element (the element find was called on).                                                                                               |
| [@attrib]         | Selects all elements that have the given attribute.                                                                                                                                                                                       |
| [@attrib='value'] | Selects all elements for which the given attribute has the given value. The value cannot contain quotes.                                                                                                                                  |
| [tag]             | Selects all elements that have a child named tag. Only immediate children are supported.                                                                                                                                                  |
| [tag='text']      | Selects all elements that have a child named tag whose complete text content, including descendants, equals the given text.                                                                                                               |
| [position]        | Selects all elements that are located at the given position. The position can be either an integer (1 is the first position), the expression last() (for the last position), or a position relative to the last position (e.g. last()-1). |

### Modifying an XML File

The ElementTree.write() method can be used to save the updated document to specified file. 

In [15]:
root = ET.fromstring(xml_book)
ele = root.find('book')

ele.attrib['name'] = "John Cleese"
updated_xml = 'code/data/updated_old_book.xml'
tree.write(updated_xml)

In [16]:
with open(updated_xml) as f:
    print(f.read())

<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
	   <book part="2">newbook2</book>
      <price>44.95</price>
      <publish_date>2000-10-01</publish_date>
      <description>An in-depth look at creating applications 
      with XML.</description>
   </book>
   <book id="bk102">
      <author>Ralls, Kim</author>
      <title>Midnight Rain</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-12-16</publish_date>
      <description>A former architect battles corporate zombies, 
      an evil sorceress, and her own childhood to become queen 
      of the world.</description>
   </book>
   <book id="bk103">
      <author>Corets, Eva</author>
      <title>Maeve Ascendant</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-11-17</publish_date>
      <description>After the collapse of a nanotechnology 
      society in Engla

### XML vulnerabilities

he XML processing modules are not secure against maliciously constructed data. An attacker can abuse XML features to carry out denial of service attacks, access local files, generate network connections to other machines, or circumvent firewalls.

The following table gives an overview of the known attacks and whether the various modules are vulnerable to them. 

| kind | sax | etree | minidom | pulldom | xmlrpc |
|---------------------------|------------|------------|------------|------------|------------|
| billion laughs | Vulnerable | Vulnerable | Vulnerable | Vulnerable | Vulnerable |
| quadratic blowup | Vulnerable | Vulnerable | Vulnerable | Vulnerable | Vulnerable |
| external entity expansion | Vulnerable | Safe | Safe | Vulnerable | Safe |
| DTD retrieval | Vulnerable | Safe | Safe | Vulnerable | Safe |
| decompression bomb | Safe | Safe | Safe | Safe | Vulnerable |

> **<center>Note</center>**
> <hr>
> Please refer Python docs for more details.

## Common Errors and causes

In [85]:
xml_book = """
<?xml version="1.0"?>
<books>
    <book title="Ṛg-Veda Khilāni">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2008</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/rvkh/rvkh.htm</web_page>
    </book>    
    <book title="Ṛgveda-Saṃhitā">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2000</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/mt/rv.htm</web_page>
    </book>
</books>
"""
try:
    root = ET.fromstring(xml_book)
except Exception as e:
    print(e)

XML or text declaration not at start of entity: line 2, column 0


due to blank first line this error happens, to avoid this error remove the blank spaces from the start of string, as shown below

In [61]:
xml_book = """<?xml version="1.0"?>
<books>
    <book title="Ṛg-Veda Khilāni">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2008</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/rvkh/rvkh.htm</web_page>
    </book>    
    <book title="Ṛgveda-Saṃhitā">
        <editor>Jost Gippert</editor>
        <publication>Frankfurt: TITUS</publication>
        <year>2000</year>
        <web_page>http://titus.uni-frankfurt.de/texte/etcs/ind/aind/ved/rv/mt/rv.htm</web_page>
    </book>
</books>
"""
root = ET.fromstring(xml_book)

**Questions:**

- Can you parse a xml document made of one tag `countries` with two children "field" named `name` and `flag`? 

## Reference & Inspirations

- https://en.wikipedia.org/wiki/XML
- https://docs.python.org/3.6/library/xml.html
- https://en.wikipedia.org/wiki/Billion_laughs
- https://www.ibm.com/developerworks/library/x-hiperfparse/index.html
- https://github.com/mikekestemont/ghent1516/blob/master/Chapter%208%20-%20Parsing%20XML.ipynb 